In [12]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.metrics import classification_report
from collections import Counter

In [16]:
df = pd.read_csv("manual_labelled.csv")

In [20]:
df.head()

,topic,text,inappropriate
0,sexual_minorities,"У меня есть друзья-пидоры, нормальные вроде ре...",0
1,sexual_minorities,"В Израиле вот постоянно гей-парады устраивают,...",0
2,sexual_minorities,"Много ли девушек, которым совсем незнаком лесб...",0
3,sexual_minorities,"В Британии лечили геев химической кастрацией, ...",1
4,sexual_minorities,Если мужчина возляжет с мужчиной как с женщино...,1


In [13]:
Counter(df.topic)

Counter({'sexual_minorities': 14,
         'racism': 13,
         'health_shaming': 12,
         'body_shaming': 12,
         'sexism': 11,
         'politcs': 13,
         'religion': 10,
         'social_injustice': 10,
         'drugs': 10,
         'weapons': 10,
         'offline_crime': 10,
         'online_crime': 11,
         'gambling': 10,
         'prostitution': 10,
         'terrorism': 9,
         'slavery': 10,
         'pornography': 8,
         'suicide': 10})

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   topic          193 non-null    object
 1   text           193 non-null    object
 2   inappropriate  193 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.6+ KB


In [51]:
def get_model_predictions(model_name, device):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name);
    
    tokenized = tokenizer.batch_encode_plus(df.text.tolist(),max_length = 512,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=False)
    tokens_ids,mask = torch.tensor(tokenized['input_ids']),torch.tensor(tokenized['attention_mask']) 
    
    device = torch.device('cuda:{}'.format(device))
    model.to(device);
    
    with torch.no_grad():
        tokens_ids, mask = tokens_ids.to(model.device),mask.to(model.device)
        model_output = model(tokens_ids,mask)
        
    pred = torch.softmax(model_output['logits'], dim = 1)
    
    pred_conf = pred[:,1].cpu().tolist()
    
    pred_round = torch.argmax(pred, dim = 1)
    pred_round = pred_round.cpu().tolist()
    
    del model
    
    return pred_round, pred_conf

## Compare models

In [52]:
mn = 'Skoltech/russian-inappropriate-messages'
our_pred_round, our_pred_conf = get_model_predictions(mn,'0')

In [54]:
print(classification_report(df.inappropriate.tolist(), our_pred_round))

              precision    recall  f1-score   support

           0       0.67      0.69      0.68        95
           1       0.69      0.66      0.68        98

    accuracy                           0.68       193
   macro avg       0.68      0.68      0.68       193
weighted avg       0.68      0.68      0.68       193



In [55]:
mn = 'sismetanin/rubert-toxic-pikabu-2ch'
sm_pred_round, sm_pred_conf  = get_model_predictions(mn,'0')

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [56]:
print(classification_report(df.inappropriate.tolist(), sm_pred))

              precision    recall  f1-score   support

           0       0.64      0.68      0.66        95
           1       0.67      0.62      0.65        98

    accuracy                           0.65       193
   macro avg       0.65      0.65      0.65       193
weighted avg       0.65      0.65      0.65       193



## Get statistics

In [57]:
def print_incorrect(incorr_df, incorr_type):
    print("Should have been {}".format(incorr_type))
    current_topic = ''
    for i,el in incorr_df.iterrows():
        if el['topic'] != current_topic:
            print("="*100)
            print(el['topic'])
            current_topic = el['topic']
        print(el['text'], el['pred_conf'])

In [59]:
def get_incorrect_statistics(dataframe,predictions, predictions_conf):
    curr_df = dataframe.copy(deep=False)
    curr_df['pred'] = predictions
    curr_df['pred_conf'] = predictions_conf
    incorrect_pred_df = curr_df[curr_df['pred']!=curr_df['inappropriate']]
    incorrect_pred_df = incorrect_pred_df.sort_values(by=['topic'])
    
    incorrect_pred_df_real_1 = incorrect_pred_df[incorrect_pred_df['inappropriate']==1]
    incorrect_pred_df_real_1 = incorrect_pred_df_real_1.sort_values(by=['topic'])
    print_incorrect(incorrect_pred_df_real_1,'inappropriate')
    
    print('\n\n')
    
    incorrect_pred_df_real_0 = incorrect_pred_df[incorrect_pred_df['inappropriate']==0]
    incorrect_pred_df_real_0 = incorrect_pred_df_real_0.sort_values(by=['topic'])
    print_incorrect(incorrect_pred_df_real_0,'OK')
    
    return incorrect_pred_df
    
our_incorr = get_incorrect_statistics(df, our_pred_round, our_pred_conf )    

Should have been inappropriate
body_shaming
Да как вообще можно всерьез прислушиваться к мужчине, который красит волосы? 0.006296420935541391
gambling
Продам беспроигрышную схему игры с игровым автоматом, выигрыш каждый раз, минимум 100 рублей. 0.3826906979084015
пацаны по этому промокоду получите гарантированные выплаты даже за проигранную ставку 0.17374677956104279
А давайте перекинемся в картишки? На косарь.  0.41370800137519836
health_shaming
У тебя что психическо заболевание или что 0.03317732736468315
offline_crime
Если нужно, могу и по-мужски разобраться, отправить его в больницу месяцев на шесть 0.0017118834657594562
Избежать уголовки не сложно, главное знать, кому надо предлагать деньги 0.10896723717451096
online_crime
Купить базу телефонных номеров очень просто, поспрашивай на форуме. 0.0051035648211836815
нашел недавно базу номеров, может пригодиться продаванам 0.0018455003155395389
Пацаны откопал недавно кряк на новую винду, налетй 0.45179787278175354
Где бесплатно скачать 

In [61]:
sm_incorr = get_incorrect_statistics(df, sm_pred_round, sm_pred_conf )    

Should have been inappropriate
body_shaming
Да как вообще можно всерьез прислушиваться к мужчине, который красит волосы? 0.013123086653649807
drugs
Айкос не такие вредные как обыные сиги, но мне все равно буду дальше так же дымить 0.003815141273662448
Да мне даже в десять лет водку продавали. 0.0029016879852861166
Многие выдающиеся личности принимали вещества, и ничего, долго жили и продолжали творить, поэтому не верьте тем, кто вас запугивает.  0.058852940797805786
Так-то трава весьма безопасная, если не каждый день её употреблять.  0.002845851005986333
gambling
Продам беспроигрышную схему игры с игровым автоматом, выигрыш каждый раз, минимум 100 рублей. 0.00466263247653842
На тотализаторах реально можно заработать, я хорошо поднялся на ставках на футбольные матчи, рекомендую.  0.003003397723659873
health_shaming
Не понимаю, почему не разрешают эвтаназию для инвалидов, всё ж лучше, чем жить и мучаться 0.04269208014011383
offline_crime
Избежать уголовки не сложно, главное знать, кому н

## Compare topics

In [62]:
Counter(our_incorr.topic.tolist()).most_common(100)

[('online_crime', 7),
 ('prostitution', 6),
 ('sexual_minorities', 6),
 ('suicide', 6),
 ('offline_crime', 5),
 ('sexism', 5),
 ('slavery', 5),
 ('drugs', 3),
 ('gambling', 3),
 ('racism', 3),
 ('social_injustice', 3),
 ('body_shaming', 2),
 ('health_shaming', 2),
 ('politcs', 2),
 ('religion', 2),
 ('pornography', 1),
 ('weapons', 1)]

In [64]:
len(Counter(our_incorr.topic.tolist()))

17

In [63]:
Counter(sm_incorr.topic.tolist()).most_common(100)

[('drugs', 7),
 ('racism', 6),
 ('offline_crime', 5),
 ('online_crime', 5),
 ('politcs', 4),
 ('prostitution', 4),
 ('sexual_minorities', 4),
 ('suicide', 4),
 ('weapons', 4),
 ('body_shaming', 3),
 ('gambling', 3),
 ('religion', 3),
 ('sexism', 3),
 ('slavery', 3),
 ('social_injustice', 3),
 ('terrorism', 3),
 ('health_shaming', 2),
 ('pornography', 1)]

In [65]:
len(Counter(sm_incorr.topic.tolist()))

18